In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import math
import glob
import netCDF4 as nc
import datetime as dt
from salishsea_tools import evaltools as et, viz_tools
import gsw 
import xarray as xr
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import matplotlib.dates as mdates
import cmocean as cmo
import scipy.interpolate as sinterp
import pickle
import cmocean
import json
import f90nml
from collections import OrderedDict
from matplotlib.colors import LogNorm

def load_obs_data(year,datadir='/ocean/eolson/MEOPAR/obs/WADE/ptools_data/ecology'):
    """ Returns a dataframe containing nutrient bottle data for a given year merged with station data
    """
    dfSta=pickle.load(open(os.path.join(datadir,'sta_df.p'),'rb'))
    dfBot=pickle.load(open(os.path.join(datadir,f'Bottles_{str(year)}.p'),'rb'))
    df=pd.merge(left=dfSta,right=dfBot,how='right',
             left_on='Station',right_on='Station')
    try:
        len(df.loc[pd.isnull(df['Latitude'])]) == 0
    except:
        pass
        print('Warning!, Stations found without Latitude or Longitude value!')
    try:
        len(df) == len(dfBot)
    except:
        pass
        print(f'Warning!, Merge completed incorrectly. length of bottle data = {len(dfBot)} length of merged data = {len(df)}')
    # where no time is provided, set time to midday Pacific time = ~ 20:00 UTC
    df['UTCDateTime']=[iiD+dt.timedelta(hours=20) if pd.isnull(iiU) \
                    else iiU for iiU,iiD in \
                    zip(df['UTCDateTime'],df['Date'])]
    df.rename(columns={'UTCDateTime':'dtUTC','Latitude':'Lat','Longitude':'Lon'},inplace=True)
    df['Z']=-1*df['Z']
    df.head()
    df['NO23']=df['NO3(uM)D']+df['NO2(uM)D'] # the model does not distinguish between NO2 and NO3
    df['Amm']=df['NH4(uM)D']
    df['Si']=df['SiOH4(uM)D']
    df['Year']=[ii.year for ii in df['dtUTC']]
    df['YD']=et.datetimeToYD(df['dtUTC'])
    return(df)
    
def load_CTD_data(year,datadir='/ocean/eolson/MEOPAR/obs/WADE/ptools_data/ecology'):
    """ Returns a dataframe containing CTD data for a given year merged with station data
    """
    dfSta=pickle.load(open(os.path.join(datadir,'sta_df.p'),'rb'))
    dfCTD0=pickle.load(open(os.path.join(datadir,f'Casts_{str(year)}.p'),'rb'))
    dfCTD=pd.merge(left=dfSta,right=dfCTD0,how='right',
             left_on='Station',right_on='Station')
    try:
        dfCTD.groupby(['Station','Year','YD','Z']).count()==[1]
    except:
        pass
        print('Only one cast per CTD station per day')
    # where no time is provided, set time to midday Pacific time = ~ 20:00 UTC
    dfCTD['dtUTC']=[iiD+dt.timedelta(hours=20) for iiD in dfCTD['Date']] #Does this mean it also has that flaw where we are not sure when the data was collected?
    dfCTD.rename(columns={'Latitude':'Lat','Longitude':'Lon'},inplace=True)
    dfCTD['Z']=-1*dfCTD['Z']
    # Calculate Absolute (Reference) Salinity (g/kg) and Conservative Temperature (deg C) from 
    # Salinity (psu) and Temperature (deg C):
    press=gsw.p_from_z(-1*dfCTD['Z'],dfCTD['Lat'])
    dfCTD['SA']=gsw.SA_from_SP(dfCTD['Salinity'],press,
                           dfCTD['Lon'],dfCTD['Lat'])
    dfCTD['CT']=gsw.CT_from_t(dfCTD['SA'],dfCTD['Temperature'],press)
    dfCTD['Year']=[ii.year for ii in dfCTD['dtUTC']]
    dfCTD['YD']=et.datetimeToYD(dfCTD['dtUTC'])
    return(dfCTD)

In [8]:
maxproc=4
spath='/home/sallen/202007/202007C-p3/'
saveloc='/ocean/kflanaga/MEOPAR/202007_grid_data/'
dirname='HC202007'
year=2016

#Defining the locations 
plist=['Hoodsport','Twanoh','DabobBay','PointWells','CarrInlet','Hansville']
varNameDict={'Hoodsport':'Hoodsport','Twanoh':'Twanoh','DabobBay':'DabobBay', 'PointWells':'PointWells',
             'CarrInlet':'CarrInlet', 'Hansville':'Hansville'}

#defining the timespan and time resoltuon
t0=dt.datetime(year,1,1)
tm1=dt.datetime(year,11,15)
tm2=dt.datetime(year,11,16)
te=dt.datetime(year,12,31)
fdur=1 # length of each results file in days

#defining the model variables to extract from ptrc
evars=('votemper','vosaline')

def setup():
    ffmt='%Y%m%d'
    stencilp='SalishSea_1d_{0}_{1}_grid_T_{2}-{2}.nc'
    fnum=int(((te-t0).days+1)/fdur)#fnum=18 # number of results files per run
    runlen=fdur*fnum # length of run in days
    fnames={'grid_T':dict(),'tempBase':dict()}
    iits=t0
    ind=0
    while iits<=tm1:
        iite=iits+dt.timedelta(days=(fdur-1)) 
        iitn=iits+dt.timedelta(days=fdur)
        try:
            iifstr=glob.glob(spath+stencilp.format(t0.strftime(ffmt),te.strftime(ffmt),iits.strftime(ffmt),iite.strftime(ffmt)),recursive=True)[0]
            fnames['grid_T'][ind]=iifstr
        except:
            print('file does not exist: '+spath+stencilp.format(iits.strftime(ffmt),iite.strftime(ffmt)))
            raise
        fnames['tempBase'][ind]=saveloc+'temp/grid'+iits.strftime(ffmt)+'_'+iite.strftime(ffmt)
        iits=iitn
        ind=ind+1
    while iits<=te:
        iite=iits+dt.timedelta(days=(fdur-1)) 
        iitn=iits+dt.timedelta(days=fdur)
        try:
            iifstr=glob.glob(spath+stencilp.format(tm2.strftime(ffmt),te.strftime(ffmt),iits.strftime(ffmt),iite.strftime(ffmt)),recursive=True)[0]
            fnames['grid_T'][ind]=iifstr
        except:
            print('file does not exist: '+spath+stencilp.format(iits.strftime(ffmt),iite.strftime(ffmt)))
            raise
        fnames['tempBase'][ind]=saveloc+'temp/grid'+iits.strftime(ffmt)+'_'+iite.strftime(ffmt)
        iits=iitn
        ind=ind+1
    return spath,fnum,runlen,fnames
 

In [9]:
if __name__ == "__main__":
    spath,fnum,runlen,fnames=setup();
    print('done setup')

done setup


In [10]:
fnames

{'grid_T': {0: '/home/sallen/202007/202007C-p3/SalishSea_1d_20160101_20161231_grid_T_20160101-20160101.nc',
  1: '/home/sallen/202007/202007C-p3/SalishSea_1d_20160101_20161231_grid_T_20160102-20160102.nc',
  2: '/home/sallen/202007/202007C-p3/SalishSea_1d_20160101_20161231_grid_T_20160103-20160103.nc',
  3: '/home/sallen/202007/202007C-p3/SalishSea_1d_20160101_20161231_grid_T_20160104-20160104.nc',
  4: '/home/sallen/202007/202007C-p3/SalishSea_1d_20160101_20161231_grid_T_20160105-20160105.nc',
  5: '/home/sallen/202007/202007C-p3/SalishSea_1d_20160101_20161231_grid_T_20160106-20160106.nc',
  6: '/home/sallen/202007/202007C-p3/SalishSea_1d_20160101_20161231_grid_T_20160107-20160107.nc',
  7: '/home/sallen/202007/202007C-p3/SalishSea_1d_20160101_20161231_grid_T_20160108-20160108.nc',
  8: '/home/sallen/202007/202007C-p3/SalishSea_1d_20160101_20161231_grid_T_20160109-20160109.nc',
  9: '/home/sallen/202007/202007C-p3/SalishSea_1d_20160101_20161231_grid_T_20160110-20160110.nc',
  10: '/ho